In [ ]:
# Install required libraries
# !pip install torch
# !pip install torchvision
# !pip install kaggle
# !pip install fastapi

In [28]:
# Import required libraries
import torch
import torch.nn as nn
import torch.optim as optim
import time

In [43]:
# Custom CNN for Eye Disease Classification
class EyeDiseaseClassifier(nn.Module):
    def __init__(self, num_classes=5):
        super(EyeDiseaseClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

1. Define the transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

# 2. Load the dataset (e.g., APTOS 2019 dataset or any similar dataset)
dataset15 = datasets.ImageFolder(root='data/train15/', transform=transform)
len(dataset15)

35126

In [45]:
# 3. Split the dataset into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 3. Split the dataset into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 4. Create DataLoader objects for training and validation
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 5. Create a dictionary of the dataloaders
dataloaders = {
    'train': train_loader,
    'val': val_loader
}

In [46]:
# 6. Define the model, loss function, and optimizer
model = EyeDiseaseClassifier(num_classes=5)  # Example: 5 classes for eye disease classification
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for classification tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

In [47]:
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_metrics(outputs, labels):
    """Calculate precision, recall, and F1-score for a batch."""
    _, preds = torch.max(outputs, 1)  # Get predicted class
    preds = preds.cpu().numpy()  # Convert to numpy for sklearn
    labels = labels.cpu().numpy()  # Convert to numpy for sklearn
    
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)

    return precision, recall, f1

# 7. Training loop function
def train_model(model, dataloaders, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        
        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total_samples = 0
        total_precision, total_recall, total_f1 = 0, 0, 0  # Track metrics

        for inputs, labels in dataloaders['train']:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(torch.max(outputs, 1)[1] == labels).item()
            total_samples += labels.size(0)

            # Calculate precision, recall, and F1-score for the batch
            precision, recall, f1 = calculate_metrics(outputs, labels)
            total_precision += precision * inputs.size(0)
            total_recall += recall * inputs.size(0)
            total_f1 += f1 * inputs.size(0)

        epoch_loss = running_loss / total_samples
        epoch_acc = running_corrects / total_samples
        epoch_precision = total_precision / total_samples
        epoch_recall = total_recall / total_samples
        epoch_f1 = total_f1 / total_samples

        print(f'Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}, '
              f'Precision: {epoch_precision:.4f}, Recall: {epoch_recall:.4f}, F1-Score: {epoch_f1:.4f}')

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        val_samples = 0
        val_precision, val_recall, val_f1 = 0, 0, 0

        with torch.no_grad():
            for inputs, labels in dataloaders['val']:
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(torch.max(outputs, 1)[1] == labels).item()
                val_samples += labels.size(0)

                # Calculate precision, recall, and F1-score for the batch
                precision, recall, f1 = calculate_metrics(outputs, labels)
                val_precision += precision * inputs.size(0)
                val_recall += recall * inputs.size(0)
                val_f1 += f1 * inputs.size(0)

        val_loss /= val_samples
        val_acc = val_corrects / val_samples
        val_precision /= val_samples
        val_recall /= val_samples
        val_f1 /= val_samples

        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}, '
              f'Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1-Score: {val_f1:.4f}')

# Train the model
train_model(model, dataloaders, criterion, optimizer, 2)


Epoch 1/2
Training Loss: 0.8797, Accuracy: 0.7353, Precision: 0.5478, Recall: 0.7353, F1-Score: 0.6262
Validation Loss: 0.9205, Accuracy: 0.7252, Precision: 0.5318, Recall: 0.7252, F1-Score: 0.6120
Epoch 2/2
Training Loss: 0.8621, Accuracy: 0.7372, Precision: 0.5497, Recall: 0.7372, F1-Score: 0.6281
Validation Loss: 0.8771, Accuracy: 0.7252, Precision: 0.5318, Recall: 0.7252, F1-Score: 0.6120


In [48]:
# Save the state_dict (model parameters)
torch.save(model.state_dict(), "v1.2_vgg16_model")
print("v1.2_vgg16_model")

v1.3vgg16_model.pth
